In [14]:
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder \
    .appName("AnimeQualityAnalysis") \
    .getOrCreate()

print("Sesión de Spark iniciada")

Sesión de Spark iniciada


25/04/03 02:00:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [15]:
# Ruta al archivo CSV (modifica la ruta según corresponda)
input_path = "../../tests/vshort-final_animedataset.csv"

# Cargar el CSV en un DataFrame
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .load(input_path)

# Crear una vista temporal para poder utilizar Spark SQL
df.createOrReplaceTempView("anime")

# Mostrar algunas filas para verificar la carga
df.show(5, truncate=False)

+--------+--------+--------+-------+------+-------------+----+------+-----+---------+------+----------+---------------------------------------------------------------+
|username|anime_id|my_score|user_id|gender|title        |type|source|score|scored_by|rank  |popularity|genre                                                          |
+--------+--------+--------+-------+------+-------------+----+------+-----+---------+------+----------+---------------------------------------------------------------+
|karthiga|21      |9       |2255153|Female|One Piece    |TV  |Manga |8.54 |423868   |91.0  |35        |Action, Adventure, Comedy, Super Power, Drama, Fantasy, Shounen|
|karthiga|59      |7       |2255153|Female|Chobits      |TV  |Manga |7.53 |175388   |1546.0|188       |Sci-Fi, Comedy, Drama, Romance, Ecchi, Seinen                  |
|karthiga|74      |7       |2255153|Female|Gakuen Alice |TV  |Manga |7.77 |33244    |941.0 |1291      |Comedy, School, Shoujo, Super Power                      

In [16]:
# Consulta SQL: Filtra filas donde 'source' y 'score' no sean nulos,
# convierte 'score' a tipo float, agrupa por 'source' y calcula el promedio,
# ordenando el resultado en forma descendente.
query = """
SELECT source, AVG(CAST(score AS FLOAT)) AS avg_score
FROM anime
WHERE source IS NOT NULL AND score IS NOT NULL
GROUP BY source
ORDER BY source ASC
"""

# Ejecutar la consulta SQL
result = spark.sql(query)

# Mostrar el resultado
result.show(truncate=False)

+------------+------------------+
|source      |avg_score         |
+------------+------------------+
|4-koma manga|7.370952333722796 |
|Game        |7.240833401679993 |
|Light novel |7.7221848383671094|
|Manga       |7.773083333174387 |
|Music       |7.190000057220459 |
|Novel       |7.701379290942488 |
|Original    |7.6814285599141625|
|Other       |7.588888857099745 |
|Unknown     |6.778695707735808 |
|Visual novel|7.223617066728308 |
|Web manga   |8.149166584014893 |
+------------+------------------+



In [17]:
spark.stop()